In [1]:
import pandas as pd
import numpy as np
import pickle
import json
import datetime
import ipywidgets as widgets
from IPython.display import display, HTML, Javascript, clear_output

# Configurar Pandas para mostrar todas las columnas
pd.set_option('display.max_columns', None)

In [2]:
# Cargar el ID_Usuario desde el archivo JSON
with open('user_id.json', 'r') as f:
    user_id = json.load(f)['ID_Usuario']

# Cargar el modelo desde el archivo
with open('recomendadorknn.pkl', 'rb') as file:
    recomendadorknn = pickle.load(file)

# Leer el archivo CSV
dfusuarios = pd.read_csv('usuarios.csv')

# Extraer datos del usuario
usuario = dfusuarios[dfusuarios['ID_Usuario'] == user_id]

# Extraer los valores de los atributos de interés y convertirlos en una lista
arreglo_perfil = usuario.loc[:, 'U_Accion':'U_Western'].values.flatten().tolist()
arreglo_perfil = np.round(arreglo_perfil, 2)

In [3]:
# Variables globales
peliculas_vistas = 0
peliculas_mostradas = 0
index_pelicula = 0

# Widgets de la interfaz
output = widgets.Output()
boton_si = widgets.Button(description="Sí")
boton_no = widgets.Button(description="No")
boton_continuar = widgets.Button(description="Continuar", disabled=True)

In [4]:
def clasificacion_a_edad(clasificacion):
    if clasificacion in ('G', 'PG', 'ATP', 'ATPR', 'A', 'AA'):
        return 0
    elif clasificacion == 'B':
        return 12
    elif clasificacion in ('PG-13', 'P-13', 'P-13R'):
        return 13
    elif clasificacion == 'B-15':
        return 15
    elif clasificacion in ('P-16', 'P16R'):
        return 16
    elif clasificacion == 'R':
        return 17
    elif clasificacion in('NC-17', 'C', 'P-18'):
        return 18
    else:
        return 21  # Para clasificaciones desconocidas o no estándar, se asume una edad adulta

In [5]:
# Filtrar películas basadas en la edad del usuario y la clasificación
def filtrar_peliculas_por_edad(peliculas, edad_usuario):
    return peliculas[peliculas['Clasificacion'].apply(clasificacion_a_edad) <= edad_usuario]

In [6]:
def predict_movies(user_profile, movies_data):
    # Preparar los inputs para el modelo
    inputs = [np.concatenate([user_profile, movies_data.iloc[i, 4:-1].values, [movies_data.iloc[i, -1]]]) for i in range(len(movies_data))]
    
    # Imprimir los primeros inputs para visualización
    #print("Ejemplo de inputs enviados al modelo:")
    #for input_example in inputs[:10]:  # Cambia el 5 a otro número si deseas ver más ejemplos
    #    print(input_example)
    
    # Hacer las predicciones
    predictions = recomendadorknn.predict_proba(inputs)
    movies_data['probabilidad_gusto'] = [p[1] for p in predictions]
    
    # Ordenar las películas por la probabilidad de que al usuario le gusten
    sorted_movies = movies_data.sort_values(by='probabilidad_gusto', ascending=False)
    return sorted_movies

In [7]:
def mostrar_pelicula():
    global index_pelicula, peliculas_mostradas

    if index_pelicula >= len(peliculas_ajuste_perfil) or peliculas_mostradas >= 15:
        finalizar_ajuste()
        boton_continuar.disabled = False
        return

    row = peliculas_ajuste_perfil.iloc[index_pelicula]
    
    with output:
        clear_output()
        print(f"Ajustaremos tus preferencias con las siguientes películas")
        print(f"\n¿Has visto '{row['Titulo']}' ({row['O_Titulo']})?")
        generos_pelicula = ', '.join([genre.split('_')[1] for genre in peliculas_ajuste_perfil.columns[4:-1] if row[genre] == 1])
        print(f"Géneros: {generos_pelicula}")
        print(f"Calificación: {row['Calificacion']*100:.1f}%")
        duracion_pelicula = "Desconocida" if row['Duracion'] == 0 else f"{row['Duracion']} minutos"
        print(f"Duración: {duracion_pelicula}")
        
        display(boton_si, boton_no, boton_continuar)
    
    peliculas_mostradas += 1
    if peliculas_mostradas >= 5:
        boton_continuar.disabled = False

def ajustar_perfil(response, row):
    global arreglo_perfil, dfusuarios, user_id

    adjustments = [-0.1, -0.05, 0.05, 0.1]
    genres = peliculas_ajuste_perfil.columns[4:-1]

    for i, genre in enumerate(genres):
        if row[genre] == 1:
            arreglo_perfil[i] = max(0, min(1, arreglo_perfil[i] + adjustments[response - 1]))

    columnas = ['U_Accion', 'U_Aventura', 'U_Animacion', 'U_Comedia', 
        'U_Crimen', 'U_Documental', 'U_Drama', 'U_Familiar', 'U_Fantasia', 
        'U_Historia', 'U_Horror', 'U_Musical', 'U_Misterio', 'U_Romance', 
        'U_Ciencia_Ficcion', 'U_Suspense', 'U_Guerra', 'U_Western']

    dfusuarios.loc[dfusuarios['ID_Usuario'] == user_id, columnas] = arreglo_perfil
    dfusuarios.to_csv(r"usuarios.csv", index=False)
    
    with output:
        clear_output()
        print("Perfil actualizado:")
        generos = [genre.split('_')[1] for genre in genres]
        for i, valor in enumerate(arreglo_perfil):
            print(f"{generos[i]}: {valor:.2f}")

def on_boton_si_clicked(b):
    global peliculas_vistas, index_pelicula
    
    peliculas_vistas += 1
    
    row = peliculas_ajuste_perfil.iloc[index_pelicula]
    
    with output:
        clear_output()
        print("¿Qué te pareció la película?")
        display(boton_odie, boton_no_me_gusto, boton_me_gusto, boton_ame)
        
    index_pelicula += 1

def on_boton_no_clicked(b):
    global index_pelicula

    index_pelicula += 1
    mostrar_pelicula()

def on_boton_continuar_clicked(b):
    finalizar_ajuste()
    with output:
        with open('user_id.json', 'w') as f:
            json.dump({'ID_Usuario': user_id}, f)
        # Redirigir al menu principal
        display(Javascript('window.location.href = "home.ipynb";'))

def finalizar_ajuste():
    with output:
        clear_output()
        print("El ajuste de tu perfil ha concluido. ¡Ahora podemos recomendarte películas en cartelera que seguramente te encantarán!")
        display(boton_continuar)  # Continuar mostrando el botón

def on_boton_odie_clicked(b):
    global index_pelicula
    ajustar_perfil(1, peliculas_ajuste_perfil.iloc[index_pelicula - 1])
    mostrar_pelicula()

def on_boton_no_me_gusto_clicked(b):
    global index_pelicula
    ajustar_perfil(2, peliculas_ajuste_perfil.iloc[index_pelicula - 1])
    mostrar_pelicula()

def on_boton_me_gusto_clicked(b):
    global index_pelicula
    ajustar_perfil(3, peliculas_ajuste_perfil.iloc[index_pelicula - 1])
    mostrar_pelicula()

def on_boton_ame_clicked(b):
    global index_pelicula
    ajustar_perfil(4, peliculas_ajuste_perfil.iloc[index_pelicula - 1])
    mostrar_pelicula()



In [8]:
# Lista de géneros de películas
generos = ['Acción', 'Aventura', 'Animación', 'Comedia', 'Crimen', 'Documental',
           'Drama', 'Familiar', 'Fantasía', 'Historia', 'Horror', 'Musical',
           'Misterio', 'Romance', 'Ciencia Ficción', 'Suspense', 'Guerra', 'Western']

In [9]:
# Obtener la información del usuario
user = dfusuarios[dfusuarios['ID_Usuario'] == user_id].iloc[0]

nombre = user['Nombre']
anio_nac = user['Anio_nac']
mes_nac = user['Mes_nac']
dia_nac = user['Dia_nac']

# Calcular la edad del usuario
fecha_nacimiento = datetime.date(anio_nac, mes_nac, dia_nac)
hoy = datetime.date.today()
edad_usuario = hoy.year - fecha_nacimiento.year - ((hoy.month, hoy.day) < (fecha_nacimiento.month, fecha_nacimiento.day))


In [10]:
#Se da lectura del dataset de películas para ajustar el perfil del usuario
peliculas_anteriores = pd.read_csv(r"peliculasanteriores.csv")
#peliculas_anteriores.head(

peliculas_anteriores = filtrar_peliculas_por_edad(peliculas_anteriores, edad_usuario)
#peliculas_anteriores.head()

In [11]:
# Obtener las top 15 películas recomendadas
peliculas_ajuste_perfil = predict_movies(arreglo_perfil, peliculas_anteriores).head(15)


In [12]:
boton_si.on_click(on_boton_si_clicked)
boton_no.on_click(on_boton_no_clicked)
boton_continuar.on_click(on_boton_continuar_clicked)

boton_odie = widgets.Button(description="¡La odié!")
boton_no_me_gusto = widgets.Button(description="No me gustó")
boton_me_gusto = widgets.Button(description="Sí me gustó")
boton_ame = widgets.Button(description="¡La amé!")

boton_odie.on_click(on_boton_odie_clicked)
boton_no_me_gusto.on_click(on_boton_no_me_gusto_clicked)
boton_me_gusto.on_click(on_boton_me_gusto_clicked)
boton_ame.on_click(on_boton_ame_clicked)

display(output)
mostrar_pelicula()

Output()

In [13]:
#peliculas_ajuste_perfil.head(15)

,Titulo,O_Titulo,Clasificacion,Duracion,P_Accion,P_Aventura,P_Animacion,P_Comedia,P_Crimen,P_Documental,P_Drama,P_Familiar,P_Fantasia,P_Historia,P_Horror,P_Musical,P_Misterio,P_Romance,P_Ciencia Ficcion,P_Suspense,P_Guerra,P_Western,Calificacion,probabilidad_gusto
20,Perfect Days,Perfect Days,PG,124,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.79,0.781513
12,El color púrpura,The Color Purple,PG-13,141,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.70,0.764706
7,Vidas pasadas,Past Lives,PG-13,105,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0.78,0.697479
21,La sala de profesores,The Teachers' Lounge,PG-13,98,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0.73,0.672269
26,Flamin' Hot: El sabor que cambió la historia,Flamin' Hot,PG-13,99,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.79,0.663866
25,Indiana Jones y el dial del destino,Indiana Jones and the Dial of Destiny,PG-13,154,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.66,0.630252
10,Rustin,Rustin,PG-13,106,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.67,0.621849
11,Nyad,Nyad,PG-13,121,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.71,0.621849
28,Misión imposible: sentencia mortal parte uno,Mission: Impossible – Dead Reckoning Part One,PG-13,163,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.76,0.571429
31,Golda,Golda,PG-13,100,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0.64,0.563025


In [14]:
#print("Ajuste peliculas")

In [15]:
#arreglo_perfil

array([1.  , 0.1 , 0.95, 0.05, 0.9 , 0.  , 0.8 , 0.  , 0.8 , 0.55, 0.5 ,
       0.5 , 0.5 , 0.5 , 0.55, 0.55, 0.55, 0.5 ])

In [16]:
#dfusuarios.head()

,ID_Usuario,Nombre,Password,Sexo,Anio_nac,Mes_nac,Dia_nac,U_Accion,U_Aventura,U_Animacion,U_Comedia,U_Crimen,U_Documental,U_Drama,U_Familiar,U_Fantasia,U_Historia,U_Horror,U_Musical,U_Misterio,U_Romance,U_Ciencia_Ficcion,U_Suspense,U_Guerra,U_Western
0,100001,Ariel,ariel,1,1991,3,3,0.15,0.15,0.20,0.35,0.2,0.2,0.35,0.20,0.2,0.20,0.2,0.2,0.2,1.0,0.35,0.20,0.20,0.2
1,100002,Marco,marco,1,1996,4,1,0.50,0.60,0.50,0.50,0.5,0.6,0.90,0.55,0.5,0.70,0.5,0.5,0.5,0.5,0.50,0.55,0.50,0.5
2,100003,Ruben,ruben,1,2010,5,10,1.00,0.10,0.95,0.05,0.9,0.0,0.80,0.00,0.8,0.55,0.5,0.5,0.5,0.5,0.55,0.55,0.55,0.5
